this script is for converting the matsim travel times from aurore to an osm file which should be used as a "car" network in r5.

it is an alternative approach to properly matching the travel time onto the snman street graph.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import geopandas as gpd
import shapely as shp

import snman
from snman.constants import *
from snman import osmnx_customized as oxc

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

#CRS_internal = 29119    # for Boston
#CRS_internal = 32216    # for Chicago
CRS_internal = 2056      # for Zurich
CRS_for_export = 4326
oxc.settings.useful_tags_way = OSM_TAGS

In [2]:
tt = pd.read_csv(
    os.path.join(outputs_path, '100pct_do_not_trust_the_numbers.csv')
)

tt

,LinkId,Freespeed,From_x,From_y,To_x,To_y,car_0,car_1,car_2,car_3,...,car_15,car_16,car_17,car_18,car_19,car_20,car_21,car_22,car_23,Unnamed: 30
0,3640,5.811348,47.327088,8.381223,47.326734,8.380054,22,22,22,18,...,18,18,18,22,22,18,22,22,18,NaN
1,3638,3.676141,47.499187,8.543563,47.499622,8.544628,25,25,25,25,...,25,10,28,14,1,1,25,28,28,NaN
2,3630,3.876162,47.490902,8.556413,47.492244,8.556424,43,43,43,43,...,47,43,47,43,47,43,43,43,43,NaN
3,3634,3.910868,47.528348,8.709041,47.527457,8.706912,52,52,48,52,...,48,52,48,52,52,52,52,52,52,NaN
4,3635,3.565796,47.349795,7.986803,47.350459,7.986701,24,20,24,20,...,24,24,24,24,24,24,24,24,20,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129345,36257,3.875624,47.333018,8.305315,47.333562,8.303478,42,42,42,42,...,46,42,46,42,42,42,42,42,42,NaN
129346,36256,3.791716,47.536786,8.728047,47.536773,8.729726,33,33,33,33,...,37,37,37,33,33,37,37,33,33,NaN
129347,132450,3.604422,47.395599,8.401946,47.396158,8.401275,22,22,22,22,...,22,26,26,26,26,26,26,26,26,NaN
129348,3629,3.912491,47.349966,7.987588,47.351453,7.988677,53,49,49,49,...,53,53,53,53,53,53,49,53,49,NaN


In [3]:
import pyproj

transformer = pyproj.Transformer.from_crs(4326, 2056, always_xy=True)

tt['u'] = tt.apply(
    lambda row: shp.Point(
        *transformer.transform(row['From_y'], row['From_x'])
    ),
    axis=1
)
tt['v'] = tt.apply(
    lambda row: shp.Point(
        *transformer.transform(row['To_y'], row['To_x'])
    ),
    axis=1
)

tt['geometry'] = tt.apply(
    lambda row: shp.LineString([row['u'], row['v']]),
    axis=1
)

tt = gpd.GeoDataFrame(tt, crs=2056)

tt

,LinkId,Freespeed,From_x,From_y,To_x,To_y,car_0,car_1,car_2,car_3,...,car_18,car_19,car_20,car_21,car_22,car_23,Unnamed: 30,u,v,geometry
0,3640,5.811348,47.327088,8.381223,47.326734,8.380054,22,22,22,18,...,22,22,18,22,22,18,NaN,POINT (2671251.845845772 1242229.8898536668),POINT (2671163.959879272 1242189.490882411),"LINESTRING (2671251.846 1242229.890, 2671163.9..."
1,3638,3.676141,47.499187,8.543563,47.499622,8.544628,25,25,25,25,...,14,1,1,25,28,28,NaN,POINT (2683253.1287553683 1261522.6565044345),POINT (2683332.685601618 1261572.1175206336),"LINESTRING (2683253.129 1261522.657, 2683332.6..."
2,3630,3.876162,47.490902,8.556413,47.492244,8.556424,43,43,43,43,...,43,47,43,43,43,43,NaN,POINT (2684234.370337359 1260615.27174875),POINT (2684233.088362707 1260764.5139727283),"LINESTRING (2684234.370 1260615.272, 2684233.0..."
3,3634,3.910868,47.528348,8.709041,47.527457,8.706912,52,52,48,52,...,52,52,52,52,52,52,NaN,POINT (2695667.809590758 1264953.2251318556),POINT (2695509.07146254 1264851.620114912),"LINESTRING (2695667.810 1264953.225, 2695509.0..."
4,3635,3.565796,47.349795,7.986803,47.350459,7.986701,24,20,24,20,...,24,24,24,24,24,20,NaN,POINT (2641420.2796023902 1244471.3832419636),POINT (2641412.0728649325 1244545.1076169135),"LINESTRING (2641420.280 1244471.383, 2641412.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129345,36257,3.875624,47.333018,8.305315,47.333562,8.303478,42,42,42,42,...,42,42,42,42,42,42,NaN,POINT (2665506.873350445 1242823.0753600928),POINT (2665367.3896172764 1242882.0115964273),"LINESTRING (2665506.873 1242823.075, 2665367.3..."
129346,36256,3.791716,47.536786,8.728047,47.536773,8.729726,33,33,33,33,...,33,33,37,37,33,33,NaN,POINT (2697083.4785140203 1265914.7378848481),POINT (2697209.8841212946 1265915.348891505),"LINESTRING (2697083.479 1265914.738, 2697209.8..."
129347,132450,3.604422,47.395599,8.401946,47.396158,8.401275,22,22,22,22,...,26,26,26,26,26,26,NaN,POINT (2672724.7787943827 1249865.5624462888),POINT (2672673.3409902174 1249927.0092824404),"LINESTRING (2672724.779 1249865.562, 2672673.3..."
129348,3629,3.912491,47.349966,7.987588,47.351453,7.988677,53,49,49,49,...,53,53,53,49,53,49,NaN,POINT (2641479.4981227415 1244490.8093179928),POINT (2641560.6089161052 1244656.6295373263),"LINESTRING (2641479.498 1244490.809, 2641560.6..."


In [4]:
nd = pd.concat([tt['u'], tt['v']]).reset_index().rename(columns={0: 'geometry'})
nd['osmid'] = pd.factorize(nd['geometry'])[0]
nd[['x', 'y']] = nd.apply(
    lambda row: (row['geometry'].x, row['geometry'].y),
    axis=1,
    result_type='expand'
)
nd.drop(columns=['index'], inplace=True)
nd.drop_duplicates(inplace=True)
nd.set_index('osmid', inplace=True)
nd = gpd.GeoDataFrame(nd, geometry='geometry', crs=2056)

nd

,geometry,x,y
osmid,,,
0,POINT (2671251.846 1242229.890),2.671252e+06,1.242230e+06
1,POINT (2683253.129 1261522.657),2.683253e+06,1.261523e+06
2,POINT (2684234.370 1260615.272),2.684234e+06,1.260615e+06
3,POINT (2695667.810 1264953.225),2.695668e+06,1.264953e+06
4,POINT (2641420.280 1244471.383),2.641420e+06,1.244471e+06
...,...,...,...
58371,POINT (2655513.784 1248750.727),2.655514e+06,1.248751e+06
58372,POINT (2695766.749 1260686.433),2.695767e+06,1.260686e+06
58373,POINT (2696185.568 1245286.771),2.696186e+06,1.245287e+06


In [5]:
tt = (pd.merge(tt, nd.reset_index()[['osmid', 'geometry']], how='left', left_on='u', right_on='geometry', suffixes=['', '_right'])
    .drop(columns=['u', 'geometry_right'])
    .rename(columns={'osmid': 'u'}))
tt = (pd.merge(tt, nd.reset_index()[['osmid', 'geometry']], how='left', left_on='v', right_on='geometry', suffixes=['', '_right'])
    .drop(columns=['v', 'geometry_right'])
    .rename(columns={'osmid': 'v'}))

tt.drop(columns=['From_x', 'From_y', 'To_x', 'To_y'], inplace=True)

tt

,LinkId,Freespeed,car_0,car_1,car_2,car_3,car_4,car_5,car_6,car_7,...,car_18,car_19,car_20,car_21,car_22,car_23,Unnamed: 30,geometry,u,v
0,3640,5.811348,22,22,22,18,18,22,18,18,...,22,22,18,22,22,18,NaN,"LINESTRING (2671251.846 1242229.890, 2671163.9...",0,17952
1,3638,3.676141,25,25,25,25,25,25,1,7,...,14,1,1,25,28,28,NaN,"LINESTRING (2683253.129 1261522.657, 2683332.6...",1,13421
2,3630,3.876162,43,43,43,43,47,47,47,47,...,43,47,43,43,43,43,NaN,"LINESTRING (2684234.370 1260615.272, 2684233.0...",2,58331
3,3634,3.910868,52,52,48,52,52,52,48,48,...,52,52,52,52,52,52,NaN,"LINESTRING (2695667.810 1264953.225, 2695509.0...",3,10
4,3635,3.565796,24,20,24,20,24,24,24,24,...,24,24,24,24,24,20,NaN,"LINESTRING (2641420.280 1244471.383, 2641412.0...",4,6123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129345,36257,3.875624,42,42,42,42,42,42,46,42,...,42,42,42,42,42,42,NaN,"LINESTRING (2665506.873 1242823.075, 2665367.3...",58325,58327
129346,36256,3.791716,33,33,33,33,33,37,33,37,...,33,33,37,37,33,33,NaN,"LINESTRING (2697083.479 1265914.738, 2697209.8...",58340,29827
129347,132450,3.604422,22,22,22,22,22,22,22,22,...,26,26,26,26,26,26,NaN,"LINESTRING (2672724.779 1249865.562, 2672673.3...",58262,58042
129348,3629,3.912491,53,49,49,49,49,49,53,49,...,53,53,53,49,53,49,NaN,"LINESTRING (2641479.498 1244490.809, 2641560.6...",58339,20


In [6]:
tt['uv'] = tt.apply(lambda row: (row['u'], row['v']), axis=1)
tt.set_index('uv', inplace=True)
tt

,LinkId,Freespeed,car_0,car_1,car_2,car_3,car_4,car_5,car_6,car_7,...,car_18,car_19,car_20,car_21,car_22,car_23,Unnamed: 30,geometry,u,v
uv,,,,,,,,,,,,,,,,,,,,,
"(0, 17952)",3640,5.811348,22,22,22,18,18,22,18,18,...,22,22,18,22,22,18,NaN,"LINESTRING (2671251.846 1242229.890, 2671163.9...",0,17952
"(1, 13421)",3638,3.676141,25,25,25,25,25,25,1,7,...,14,1,1,25,28,28,NaN,"LINESTRING (2683253.129 1261522.657, 2683332.6...",1,13421
"(2, 58331)",3630,3.876162,43,43,43,43,47,47,47,47,...,43,47,43,43,43,43,NaN,"LINESTRING (2684234.370 1260615.272, 2684233.0...",2,58331
"(3, 10)",3634,3.910868,52,52,48,52,52,52,48,48,...,52,52,52,52,52,52,NaN,"LINESTRING (2695667.810 1264953.225, 2695509.0...",3,10
"(4, 6123)",3635,3.565796,24,20,24,20,24,24,24,24,...,24,24,24,24,24,20,NaN,"LINESTRING (2641420.280 1244471.383, 2641412.0...",4,6123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(58325, 58327)",36257,3.875624,42,42,42,42,42,42,46,42,...,42,42,42,42,42,42,NaN,"LINESTRING (2665506.873 1242823.075, 2665367.3...",58325,58327
"(58340, 29827)",36256,3.791716,33,33,33,33,33,37,33,37,...,33,33,37,37,33,33,NaN,"LINESTRING (2697083.479 1265914.738, 2697209.8...",58340,29827
"(58262, 58042)",132450,3.604422,22,22,22,22,22,22,22,22,...,26,26,26,26,26,26,NaN,"LINESTRING (2672724.779 1249865.562, 2672673.3...",58262,58042


In [7]:
tt['length'] = tt.apply(lambda row: row.geometry.length, axis=1)
tt

,LinkId,Freespeed,car_0,car_1,car_2,car_3,car_4,car_5,car_6,car_7,...,car_19,car_20,car_21,car_22,car_23,Unnamed: 30,geometry,u,v,length
uv,,,,,,,,,,,,,,,,,,,,,
"(0, 17952)",3640,5.811348,22,22,22,18,18,22,18,18,...,22,18,22,22,18,NaN,"LINESTRING (2671251.846 1242229.890, 2671163.9...",0,17952,96.726522
"(1, 13421)",3638,3.676141,25,25,25,25,25,25,1,7,...,1,1,25,28,28,NaN,"LINESTRING (2683253.129 1261522.657, 2683332.6...",1,13421,93.678620
"(2, 58331)",3630,3.876162,43,43,43,43,47,47,47,47,...,47,43,43,43,43,NaN,"LINESTRING (2684234.370 1260615.272, 2684233.0...",2,58331,149.247730
"(3, 10)",3634,3.910868,52,52,48,52,52,52,48,48,...,52,52,52,52,52,NaN,"LINESTRING (2695667.810 1264953.225, 2695509.0...",3,10,188.471146
"(4, 6123)",3635,3.565796,24,20,24,20,24,24,24,24,...,24,24,24,24,20,NaN,"LINESTRING (2641420.280 1244471.383, 2641412.0...",4,6123,74.179741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(58325, 58327)",36257,3.875624,42,42,42,42,42,42,46,42,...,42,42,42,42,42,NaN,"LINESTRING (2665506.873 1242823.075, 2665367.3...",58325,58327,151.423881
"(58340, 29827)",36256,3.791716,33,33,33,33,33,37,33,37,...,33,37,37,33,33,NaN,"LINESTRING (2697083.479 1265914.738, 2697209.8...",58340,29827,126.407084
"(58262, 58042)",132450,3.604422,22,22,22,22,22,22,22,22,...,26,26,26,26,26,NaN,"LINESTRING (2672724.779 1249865.562, 2672673.3...",58262,58042,80.134645


In [8]:
tt['travel_time_s'] = tt['car_7'].replace(0, 9999)
tt

,LinkId,Freespeed,car_0,car_1,car_2,car_3,car_4,car_5,car_6,car_7,...,car_20,car_21,car_22,car_23,Unnamed: 30,geometry,u,v,length,travel_time_s
uv,,,,,,,,,,,,,,,,,,,,,
"(0, 17952)",3640,5.811348,22,22,22,18,18,22,18,18,...,18,22,22,18,NaN,"LINESTRING (2671251.846 1242229.890, 2671163.9...",0,17952,96.726522,18
"(1, 13421)",3638,3.676141,25,25,25,25,25,25,1,7,...,1,25,28,28,NaN,"LINESTRING (2683253.129 1261522.657, 2683332.6...",1,13421,93.678620,7
"(2, 58331)",3630,3.876162,43,43,43,43,47,47,47,47,...,43,43,43,43,NaN,"LINESTRING (2684234.370 1260615.272, 2684233.0...",2,58331,149.247730,47
"(3, 10)",3634,3.910868,52,52,48,52,52,52,48,48,...,52,52,52,52,NaN,"LINESTRING (2695667.810 1264953.225, 2695509.0...",3,10,188.471146,48
"(4, 6123)",3635,3.565796,24,20,24,20,24,24,24,24,...,24,24,24,20,NaN,"LINESTRING (2641420.280 1244471.383, 2641412.0...",4,6123,74.179741,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(58325, 58327)",36257,3.875624,42,42,42,42,42,42,46,42,...,42,42,42,42,NaN,"LINESTRING (2665506.873 1242823.075, 2665367.3...",58325,58327,151.423881,42
"(58340, 29827)",36256,3.791716,33,33,33,33,33,37,33,37,...,37,37,33,33,NaN,"LINESTRING (2697083.479 1265914.738, 2697209.8...",58340,29827,126.407084,37
"(58262, 58042)",132450,3.604422,22,22,22,22,22,22,22,22,...,26,26,26,26,NaN,"LINESTRING (2672724.779 1249865.562, 2672673.3...",58262,58042,80.134645,22


In [9]:
tt['highway'] = 'primary'
tt

,LinkId,Freespeed,car_0,car_1,car_2,car_3,car_4,car_5,car_6,car_7,...,car_21,car_22,car_23,Unnamed: 30,geometry,u,v,length,travel_time_s,highway
uv,,,,,,,,,,,,,,,,,,,,,
"(0, 17952)",3640,5.811348,22,22,22,18,18,22,18,18,...,22,22,18,NaN,"LINESTRING (2671251.846 1242229.890, 2671163.9...",0,17952,96.726522,18,primary
"(1, 13421)",3638,3.676141,25,25,25,25,25,25,1,7,...,25,28,28,NaN,"LINESTRING (2683253.129 1261522.657, 2683332.6...",1,13421,93.678620,7,primary
"(2, 58331)",3630,3.876162,43,43,43,43,47,47,47,47,...,43,43,43,NaN,"LINESTRING (2684234.370 1260615.272, 2684233.0...",2,58331,149.247730,47,primary
"(3, 10)",3634,3.910868,52,52,48,52,52,52,48,48,...,52,52,52,NaN,"LINESTRING (2695667.810 1264953.225, 2695509.0...",3,10,188.471146,48,primary
"(4, 6123)",3635,3.565796,24,20,24,20,24,24,24,24,...,24,24,20,NaN,"LINESTRING (2641420.280 1244471.383, 2641412.0...",4,6123,74.179741,24,primary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(58325, 58327)",36257,3.875624,42,42,42,42,42,42,46,42,...,42,42,42,NaN,"LINESTRING (2665506.873 1242823.075, 2665367.3...",58325,58327,151.423881,42,primary
"(58340, 29827)",36256,3.791716,33,33,33,33,33,37,33,37,...,37,33,33,NaN,"LINESTRING (2697083.479 1265914.738, 2697209.8...",58340,29827,126.407084,37,primary
"(58262, 58042)",132450,3.604422,22,22,22,22,22,22,22,22,...,26,26,26,NaN,"LINESTRING (2672724.779 1249865.562, 2672673.3...",58262,58042,80.134645,22,primary


In [10]:
min(tt['length'])

0.0

In [11]:
tt['maxspeed'] = 40
tt

,LinkId,Freespeed,car_0,car_1,car_2,car_3,car_4,car_5,car_6,car_7,...,car_22,car_23,Unnamed: 30,geometry,u,v,length,travel_time_s,highway,maxspeed
uv,,,,,,,,,,,,,,,,,,,,,
"(0, 17952)",3640,5.811348,22,22,22,18,18,22,18,18,...,22,18,NaN,"LINESTRING (2671251.846 1242229.890, 2671163.9...",0,17952,96.726522,18,primary,40
"(1, 13421)",3638,3.676141,25,25,25,25,25,25,1,7,...,28,28,NaN,"LINESTRING (2683253.129 1261522.657, 2683332.6...",1,13421,93.678620,7,primary,40
"(2, 58331)",3630,3.876162,43,43,43,43,47,47,47,47,...,43,43,NaN,"LINESTRING (2684234.370 1260615.272, 2684233.0...",2,58331,149.247730,47,primary,40
"(3, 10)",3634,3.910868,52,52,48,52,52,52,48,48,...,52,52,NaN,"LINESTRING (2695667.810 1264953.225, 2695509.0...",3,10,188.471146,48,primary,40
"(4, 6123)",3635,3.565796,24,20,24,20,24,24,24,24,...,24,20,NaN,"LINESTRING (2641420.280 1244471.383, 2641412.0...",4,6123,74.179741,24,primary,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(58325, 58327)",36257,3.875624,42,42,42,42,42,42,46,42,...,42,42,NaN,"LINESTRING (2665506.873 1242823.075, 2665367.3...",58325,58327,151.423881,42,primary,40
"(58340, 29827)",36256,3.791716,33,33,33,33,33,37,33,37,...,33,33,NaN,"LINESTRING (2697083.479 1265914.738, 2697209.8...",58340,29827,126.407084,37,primary,40
"(58262, 58042)",132450,3.604422,22,22,22,22,22,22,22,22,...,26,26,NaN,"LINESTRING (2672724.779 1249865.562, 2672673.3...",58262,58042,80.134645,22,primary,40


In [12]:
G = snman.street_graph.street_graph_from_gdf(nd, tt)

In [13]:
if 0:
    snman.io.export_street_graph(
        G,
        os.path.join(outputs_path, 'tt_edges.gpkg'),
        os.path.join(outputs_path, 'tt_nodes.gpkg'),
        crs=CRS_for_export
    )

In [14]:
snman.io.export_osm_xml(
    G,
    os.path.join(outputs_path, 'tt.osm'),
    tags=['LinkId', 'Freespeed', 'maxspeed', 'travel_time_s', 'length', 'highway'],
    uv_tags=True,
    overwrite_highway='primary',
    set_maxspeed_by_cost='travel_time_s',
    raw_graph=True,
    floor_maxspeed=1,
    ceil_maxspeed=120
)

snman.io.osm_to_pbf(os.path.join(outputs_path, 'tt.osm'))

FileNotFoundError: [WinError 2] The system cannot find the file specified